In [ ]:
import time
from pymysql import *
import pandas as pd
import numpy as np
import datetime

In [ ]:
conn = connect(
    host = '127.0.0.1',
    user = 'root',
    password = 'gky308219',
    db = 'paper_web',
    port = 3306,
    charset = 'utf8'
)

In [ ]:
cs = conn.cursor()
s = "select id from db_source_detail_info where source_name=%s"
cs.execute(s, test)

In [ ]:
d = cs.fetchall()

In [ ]:
d[0][0]

In [ ]:
source_type = data.loc[5]['Publication Type']
test = data.loc[5]['Source Title'] if source_type != 'C' else data.loc[5]['Conference Title']
test

In [ ]:
sql = "insert into db_source_detail_info values (null, %s, %s, %s, %s, %s, %s, %s, %s, %s, 1, %s)"

In [ ]:
val = (source_name, source_type, source_date, source_location, source_publisher, source_issn, source_eissn, source_isbn, source_abbreviation, edit_date)

In [ ]:
data = pd.read_excel('1 (1).xls', sheet_name='savedrecs', keep_default_na=False)

In [ ]:
source_type = data.loc[0]['Publication Type']
source_name = data.loc[0]['Source Title'] if source_type != 'C' else data.loc[0]['Conference Title']
source_date = ' ' if len(data.loc[0]['Conference Date']) == 0 else data.loc[0]['Conference Date']
source_location = data.loc[0]['Conference Location'] if source_type == 'C' else data.loc[0]['Addresses']
source_location = ' '.join(source_location.split(','))
source_publisher = ' '.join(data.loc[0]['Publisher'].split(','))
source_issn = data.loc[0]['ISSN']
source_eissn = data.loc[0]['eISSN']
source_isbn = data.loc[0]['ISBN']
source_abbreviation = data.loc[0]['Journal Abbreviation']
edit_date = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')

In [ ]:
source_type

In [ ]:
# 导入期刊数据
sql = "insert into db_source_detail_info values (null, %s, %s, %s, %s, %s, %s, %s, %s, %s, 1, %s)"
for i in range(len(data)):
    source_type = data.loc[i]['Publication Type']
    source_name = data.loc[i]['Source Title'] if source_type != 'C' else data.loc[i]['Conference Title']
    source_date = data.loc[i]['Conference Date']
    source_location = data.loc[i]['Conference Location'] if source_type == 'C' else data.loc[i]['Addresses']
    source_publisher = data.loc[i]['Publisher']
    source_issn = data.loc[i]['ISSN']
    source_eissn = data.loc[i]['eISSN']
    source_isbn = data.loc[i]['ISBN']
    source_abbreviation = data.loc[i]['Journal Abbreviation']
    edit_date = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    
    val = (source_name, source_type, source_date, source_location, source_publisher, source_issn, source_eissn, source_isbn, source_abbreviation, edit_date)
    cs = conn.cursor()
    try:
        cs.execute(sql, val)
    except Exception as e:
        print('插入SourceDetailInfo',e)
    
    conn.commit()

In [ ]:
# 导入文章数据
sql = "insert into db_paper_main_info values(null, %s, %s, %s, %s, %s, %s, 1, %s, %s)"
sel = "select id from db_source_detail_info where source_name=%s"
for i in range(len(data)):
    source_type = data.loc[i]['Publication Type']
    source_name = data.loc[i]['Source Title'] if source_type != 'C' else data.loc[i]['Conference Title']
    
    cs = conn.cursor()
    cs.execute(sel, source_name)
    try:
        temp = cs.fetchall()[0][0]
        paper_title = data.loc[i]['Article Title']
        paper_author = data.loc[i]['Author Full Names']
        paper_publish_year = data.loc[i]['Publication Year']
        paper_keywords = data.loc[i]['Author Keywords']
        paper_abstract = data.loc[i]['Abstract']
        paper_affiliations = data.loc[i]['Affiliations']
        edit_date = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')

        val = (paper_title, paper_author, paper_publish_year, paper_keywords, paper_abstract, paper_affiliations, edit_date, str(temp))

        try:
            cs.execute(sql, val)
        except Exception as e:
            print('插入MainInfo', e)
    except Exception as e:
        print('检索不到期刊:',source_name)


    conn.commit()

In [ ]:
# 导入其他数据
sql_detail = "insert into db_paper_detail_info values(null, %s, %s, %s, %s, %s, %s, %s, 1, %s, %s)"
sql_cited = "insert into db_paper_cited_count values(null, %s, %s, 1, %s, %s)"
sql_download = "insert into db_paper_download_count values(null, 0, 1, %s, %s)"
sql_file = "insert into db_paper_file_path values(null, null, 1, %s, %s)"
sel = "select id from db_paper_main_info where paper_title=%s and paper_publish_year=%s"

for i in range(len(data)):
    paper_title = data.loc[i]['Article Title']
    paper_publish_year = data.loc[i]['Publication Year']
    cs = conn.cursor()
    val = (paper_title, paper_publish_year)
    cs.execute(sel, val)
    
    try:
        temp = cs.fetchall()[0][0]
        
        paper_language = data.loc[i]['Language']
        paper_volume = data.loc[i]['Volume']
        paper_issue = data.loc[i]['Issue']
        paper_start_page = data.loc[i]['Start Page']
        paper_end_page = data.loc[i]['End Page']
        paper_page = str(paper_start_page) + '-' + str(paper_end_page)
        paper_doi = data.loc[i]['DOI']
        paper_ids = data.loc[i]['IDS Number']
        paper_wos = data.loc[i]['UT (Unique WOS ID)']
        
        cited_reference_count = data.loc[i]['Cited Reference Count']
        cited_times = data.loc[i]['Times Cited, All Databases']
        
        edit_date = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        
        val_detail = (paper_language, paper_volume, paper_issue, paper_page, paper_doi, paper_ids, paper_wos, edit_date, str(temp))
        val_cited = (int(cited_reference_count), int(cited_times), edit_date, str(temp))
        val_download = (edit_date, str(temp))
        val_file = (edit_date, str(temp))
        
        try:
            cs.execute(sql_detail, val_detail)
        except Exception as e:
            print('插入Detail', e)
        try:
            cs.execute(sql_cited, val_cited)
        except Exception as e:
            print('插入Cited', e)
        try:
            cs.execute(sql_download, val_download)
        except Exception as e:
            print('插入Download', e)
        try:
            cs.execute(sql_file, val_file)
        except Exception as e:
            print('插入File', e)
    except Exception as e:
        print('文章没有存入数据库：',paper_title)
        
    conn.commit()

In [ ]:
cs = conn.cursor()
try: 
    cs.execute(sql)
except Exception as e:
    print('插入source',e)


In [ ]:
source_id = cs.execute(f'select id from db_source_detail_info where source_name="{source_name}"')

In [ ]:
for i in range(len(data)):
    source_type = data.loc[i]['Publication Type']
    source_name = data.loc[i]['Source Title'] if source_type != 'C' else data.loc[i]['Conference Title']
    source_date = data.loc[i]['Conference Date']
    source_location = data.loc[i]['Conference Location'] if source_type == 'C' else data.loc[i]['Addresses']
    source_publisher = data.loc[i]['Publisher']
    source_issn = data.loc[i]['ISSN']
    source_eissn = data.loc[i]['eISSN']
    source_isbn = data.loc[i]['ISBN']
    source_abbreviation = data.loc[i]['Journal Abbreviation']

    cs = conn.cursor()
    try:
        sql = f'insert into paper_sourcedetailinfomodel(source_name,source_type,source_date,source_location,source_publisher_source_issn,source_eissn,source_isbn,source_abbrevation) values({source_name},{source_type},{source_date},{source_location},{source_publisher},{source_issn},{source_eissn},{source_isbn},{source_abbreviation}'
        cs.execute(sql)
    except Exception as e:
        print('插入source',e)
    source_id = cs.execute(f'select source_id from paper_sourcedetailinfomodel where source_name={source_name}').fetchall()

    paper_title = data.loc[i]['Article Title']
    paper_author = data.loc[i]['Author Full Names']
    paper_publish_year = data.loc[i]['Publication Year']
    paper_keywords = data.loc[i]['Author Keywords']
    paper_abstract = data.loc[i]['Abstract']
    paper_affiliations = data.loc[i]['Affiliations']

    try:
        sql = f'insert into paper_papermaininfomodel(paper_title,paper_author,paper_publisher_year,paper_keywords,paper_abstract,paper_affiliations,paper_source_id) values({paper_title},{paper_author},{paper_publish_year},{paper_keywords},{paper_abstract},{paper_affiliations},{source_id})'
        cs.execute('插入MainInfo',sql)
    except Exception as e:
        print(e)
    paper_id = cs.execute(f'select paper_id from paper_papermaininfomodel where paper_title={paper_title}').fetchall()[0]

    paper_language = data.loc[i]['Language']
    paper_volume = data.loc[i]['Volume']
    paper_issue = data.loc[i]['Issue']
    paper_start_page = data.loc[i]['Start Page']
    paper_end_page = data.loc[i]['End Page']
    paper_page = str(paper_start_page) + str(paper_end_page)
    paper_doi = data.loc[i]['DOI']
    paper_ids = data.loc[i]['IDS Number']
    paper_wos = data.loc[i]['UT (Unique WOS ID)']


    try: 
        sql = f'insert into paper_paperdetailinfomodel(paper_language,paper_volume,paper_issue,paper_page,paper_doi,paper_ids,paper_wos,paper_id_id) values({paper_language},{paper_volume},{paper_issue},{paper_page},{paper_doi},{paper_ids},{paper_wos},{paper_id})'
        cs.execute(sql)
    except Exception as e:
        print('插入DetailInfo',e)

    cited_reference_count = data.loc[i]['Cited Reference Count']
    cited_times = data.loc[i]['Times Cited, All Databases']
    try:
        sql = f'insert into paper_papercitedcountmodel(cited_reference_count,cited_times,paper_id_id) values({cited_reference_count},{cited_times},{paper_id})'
        cs.exceute(sql)
    except Exception as e:
        print('插入CitedCount',e)
    
    try:
        cs.exceute(f'insert into paper_paperdownloadcountmodel(paper_id_id) values({paper_id})')
        cs.exceute(f'insert into paper_paperfilepathmodel(paper_id_id) values({paper_id})')
    except Exception as e:
        print('最后两个',e)
        

In [ ]:
B = insert(conn, data)

In [ ]:
for i in range(len(data)):
    print('当前出版商：',data.loc[i]['Publisher'], '\n')

In [ ]:
len(data)

In [ ]:
def GetInfo(data):
    for itme in data:
        
    source_name = data['']
    source_type = data['']
    source_date = data['']
    source_location = data['']
    source_publisher = data['']
    source_issn = data['']
    source_eissn = data['']
    source_isbn = data['']
    source_abbreviation = data['']
    
    paper_title = data['']
    paper_author = data['']
    paper_publish_year = ['']
    paper_keywords = data['']
    paper_abstract = data['']
    paper_affiliations = data['']
    
    paper_language = data['']
    paper_volume = data['']
    paper_issue = data['']
    paper_page = data['']
    paper_doi = data['']
    paper_ids = data['']
    paper_wos = data['']
    
    cited_reference_count = data['']
    cited_times = data['']
    

In [ ]:
time.time()